In [1]:
import os
import sys
import yaml

In [2]:
import discord
import pandas as pd
import numpy as np
from discord.ext import commands
from Bio import Entrez
from bs4 import BeautifulSoup 
import requests


In [3]:
from pubmed_scraper import *
from bvsalud_scraper import *
from read_cred import *

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
secret_file = 'credentials.yaml'
credentials = read_secret(secret_file)
discord_token = credentials['discord_token']
discord_channel = credentials['discord_channel']

In [7]:
intents = discord.Intents.all()

In [8]:
bot = commands.Bot(
    command_prefix= "+",
    description= 'Bot',
    intents= intents
)

In [9]:
@bot.event
async def on_ready():
    print(f"Bot is ready. Connected to {len(bot.guilds)} servers.")

In [10]:
@bot.event
async def on_message(message):
    # print("message: ", message)
    # print(f"USER - {message.author} texted - {message.content}")
    await bot.process_commands(message)

In [11]:
@bot.event
async def on_member_join(member):
    welcome_channel = bot.get_channel(discord_channel)
    welcome_message = f'Bienvenido a researcher, {member.mention}!'
    await welcome_channel.send(welcome_message)

In [12]:
@bot.command()
async def hola(ctx):
    print("Command received")
    await ctx.send("Hola! \nRecibi los ultimos titulos publicados en BVSALUD y PUBMED")
    await ctx.send("\nEscribi lo siguiente: \n+pubmed palabra y/o palabras claves \n+bvsalud palabra y/o palabras claves")
    await ctx.send("\n+adios para salir")
    await ctx.send("\n+hola para comenzar de nuevo si saliste")

In [13]:
# Command to return titles 
@bot.command()
async def pubmed(ctx, *args ):
    user_query = ' '.join(args)
    print("Command received")
    try:
        df = get_df_pubmed(user_query)
        titles = [f'{index + 1}: {title}\n' for index, title 
            in enumerate(df['Title'].tolist())]
        await ctx.send(f'Estos son los ultimos titulos publicados en PUBMED')
        await ctx.send(''.join(titles))
        file_path = 'pubmed_titles.xlsx'
        df.to_excel(file_path, index= False)

        with open(file_path, 'rb') as file:
            await ctx.send(file= discord.File(file, filename='pubmed_titles.xlsx'))

        os.remove(file_path)

    except Exception as e: 
        await ctx.send(f'Ocurrio un error:{e}')     

In [14]:
# Command to return titles 
@bot.command()
async def bvsalud(ctx, *args):
    user_query = concatenate_query(args)
    print("Command received")
    #print(user_query)
    try: 
      df = get_bvsalud(user_query)
      #print(df.columns)
      titles = [f'{index + 1}: {title}\n' for index, title 
          in enumerate(df['Title'].tolist())]
      await ctx.send(f'Estos son los ultimos titulos publicados en BVSALUD')
      await ctx.send(''.join(titles))
      file_path = 'bvsalud_titles.xlsx'
      df.to_excel(file_path, index= False)

      with open(file_path, 'rb') as file:
        await ctx.send(file= discord.File(file, filename='bvsalud_titles.xlsx'))

      os.remove(file_path)
    except Exception as e:
      await ctx.send(f'Ocurrio un error:{e}') 

In [15]:
@bot.command()
async def adios(ctx):
    await ctx.send(f'Hasta luego..!')
    await bot.close()

In [16]:
# Run the bot with your bot token
bot.run(discord_token)

[2023-07-28 15:45:10] [INFO    ] discord.client: logging in using static token
[2023-07-28 15:45:12] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ef68a4961194913b8820d073712c6f86).


Bot is ready. Connected to 1 servers.
Command received
